<div style="background: #000;
            color: #FFF;
            margin: 0px;
            padding: 10px 0px 20px 0px;
            text-align: center; 
                ">
    <h1>Week 18 Lab</h1>
</div>

For today's lab, we'll be using the Marvel comic book characters data set available from [538](https://github.com/fivethirtyeight/data/tree/master/comic-characters)

The data comes from [Marvel Wikia](http://marvel.wikia.com/Main_Page) and [DC Wikia](http://dc.wikia.com/wiki/Main_Page). Characters were scraped on August 24. Appearance counts were scraped on September 2. The month and year of the first issue each character appeared in was pulled on October 6.

The data is split into two files, for DC and Marvel, respectively: `dc-wikia-data.csv` and `marvel-wikia-data.csv`. Each file has the following variables:

Variable | Definition
---|---------
`page_id` | The unique identifier for that characters page within the wikia
`name` | The name of the character
`urlslug` | The unique url within the wikia that takes you to the character
`ID` | The identity status of the character (Secret Identity, Public identity, [on marvel only: No Dual Identity])
`ALIGN` | If the character is Good, Bad or Neutral
`EYE` | Eye color of the character
`HAIR` | Hair color of the character
`SEX` | Sex of the character (e.g. Male, Female, etc.)
`GSM` | If the character is a gender or sexual minority (e.g. Homosexual characters, bisexual characters)
`ALIVE` | If the character is alive or deceased
`APPEARANCES` | The number of appareances of the character in comic books (as of Sep. 2, 2014. Number will become increasingly out of date as time goes on.)
`FIRST APPEARANCE` | The month and year of the character's first appearance in a comic book, if available
`YEAR` | The year of the character's first appearance in a comic book, if available


Importing the libraries we'll be using

In [1]:
import numpy as np 
import pandas as pd 
import datetime

Let's open the `marvel-wikia-data.csv` file and generate a dataframe. Then we'll check the top of the `df` to see what we have.

In [2]:
df = pd.read_csv("../datasets/marvel-wikia-data.csv")
df.head()

,page_id,name,urlslug,ID,ALIGN,EYE,HAIR,SEX,GSM,ALIVE,APPEARANCES,FIRST APPEARANCE,Year
0,1678,Spider-Man (Peter Parker),\/Spider-Man_(Peter_Parker),Secret Identity,Good Characters,Hazel Eyes,Brown Hair,Male Characters,NaN,Living Characters,4043.0,Aug-62,1962.0
1,7139,Captain America (Steven Rogers),\/Captain_America_(Steven_Rogers),Public Identity,Good Characters,Blue Eyes,White Hair,Male Characters,NaN,Living Characters,3360.0,Mar-41,1941.0
2,64786,"Wolverine (James \""Logan\"" Howlett)",\/Wolverine_(James_%22Logan%22_Howlett),Public Identity,Neutral Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,3061.0,Oct-74,1974.0
3,1868,"Iron Man (Anthony \""Tony\"" Stark)",\/Iron_Man_(Anthony_%22Tony%22_Stark),Public Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,2961.0,Mar-63,1963.0
4,2460,Thor (Thor Odinson),\/Thor_(Thor_Odinson),No Dual Identity,Good Characters,Blue Eyes,Blond Hair,Male Characters,NaN,Living Characters,2258.0,Nov-50,1950.0


As you can see we have a couple of issues:
* The column `names` has, in some cases, both the hero name and their secret identity. In other cases, it has the universe the hero is from (usually Earth-616). We'd like for `name` to be the superhero. If a secret identity name is available, we'd like for that to be in it's own column. We'd also like a column `origin` for the superhero's place of origin (usually Earth-616).
* The `urlslug` column is incorrect. We'd like for that to be a complete link. For example, spiderman's wikia link is https://marvel.fandom.com/wiki/Peter_Parker_(Earth-616) but if you use the `Spider-Man_(Peter_Parker)` from the `urlslug` (https://marvel.fandom.com/wiki/Spider-Man_(Peter_Parker), the page redirects to the correct page. You can choose to fix it various different ways but it should be a working link. Use "https://marvel.fandom.com/wiki/" as the beginning of your url.
* For the column `align`, we'd like for the "Characters" part to be removed. We'd like for that column to be categorical (Good, Neutral, Evil or blank).
* Same for the columns `EYE`, `HAIR`, `SEX` and `ID`. We want to drop the extraneous data and have simple categories (e.g. for `ID` we want categories ["Known","Public", "Secret", "None"] and for `SEX` we just want ["Male", "Female","Genderfluid","Agender"]. There's a way shown below that shows the various possible categories.
* Some data is in the wrong format. For example, `Year` and `APPEARANCES` are floats but they really should be ints (as you cannot have a non-int number of appearances). 
* Some data doesn't match up (like the `FIRST APPEARANCE` and `Year` columns might have not be the same year)
* Some columns are unnecessary. Some are missing data. Some rows are duplicates. Fix those as necessary.
* Some column names are written as uppercase, some are lowercase and some are mixed case. Normalize them so all the column names are in the same format.

Fix as many of these issues as you're able to. At the minimum, complete 4 of the above bulletpoints.

In [3]:
def see_possible_categories(col):
    return df.groupby(col).sum()
    
see_possible_categories('name')

,page_id,APPEARANCES,Year
name,,,
'Spinner (Earth-616),322600,0.0,2007.0
107 (Earth-616),116257,1.0,1994.0
11-Ball (Earth-616),543479,1.0,1991.0
115 (Legion Personality) (Earth-616),624448,1.0,2010.0
14 (Earth-616),116253,1.0,1994.0
...,...,...,...
Zzxz (Earth-616),250460,6.0,2009.0
Zzzax (Earth-616),1553,32.0,1973.0
"\""Spider-Girl\"" (Mutant\/Spider Clone) (Earth-616)",689359,0.0,2013.0


In [4]:
names = see_possible_categories('name')
names

,page_id,APPEARANCES,Year
name,,,
'Spinner (Earth-616),322600,0.0,2007.0
107 (Earth-616),116257,1.0,1994.0
11-Ball (Earth-616),543479,1.0,1991.0
115 (Legion Personality) (Earth-616),624448,1.0,2010.0
14 (Earth-616),116253,1.0,1994.0
...,...,...,...
Zzxz (Earth-616),250460,6.0,2009.0
Zzzax (Earth-616),1553,32.0,1973.0
"\""Spider-Girl\"" (Mutant\/Spider Clone) (Earth-616)",689359,0.0,2013.0


- The column names has, in some cases, both the hero name and their secret identity. In other cases, it has the universe the hero is from (usually Earth-616). We'd like for name to be the superhero. If a secret identity name is available, we'd like for that to be in it's own column. We'd also like a column origin for the superhero's place of origin (usually Earth-616).

In [5]:
n = pd.Series(df['name'])
n

0                  Spider-Man (Peter Parker)
1            Captain America (Steven Rogers)
2        Wolverine (James \"Logan\" Howlett)
3          Iron Man (Anthony \"Tony\" Stark)
4                        Thor (Thor Odinson)
                        ...                 
16371                     Ru'ach (Earth-616)
16372        Thane (Thanos' son) (Earth-616)
16373          Tinkerer (Skrull) (Earth-616)
16374         TK421 (Spiderling) (Earth-616)
16375                  Yologarch (Earth-616)
Name: name, Length: 16376, dtype: object

In [6]:
n.str.split(r'\((.+)\)', expand=True)

,0,1,2
0,Spider-Man,Peter Parker,
1,Captain America,Steven Rogers,
2,Wolverine,"James \""Logan\"" Howlett",
3,Iron Man,"Anthony \""Tony\"" Stark",
4,Thor,Thor Odinson,
...,...,...,...
16371,Ru'ach,Earth-616,
16372,Thane,Thanos' son) (Earth-616,
16373,Tinkerer,Skrull) (Earth-616,
16374,TK421,Spiderling) (Earth-616,


- The urlslug column is incorrect. We'd like for that to be a complete link. For example, spiderman's wikia link is https://marvel.fandom.com/wiki/Peter_Parker_(Earth-616) but if you use the Spider-Man_(Peter_Parker) from the urlslug (https://marvel.fandom.com/wiki/Spider-Man_(Peter_Parker), the page redirects to the correct page. You can choose to fix it various different ways but it should be a working link. Use "https://marvel.fandom.com/wiki/" as the beginning of your url.

In [7]:
df['urlslug']

0                    \/Spider-Man_(Peter_Parker)
1              \/Captain_America_(Steven_Rogers)
2        \/Wolverine_(James_%22Logan%22_Howlett)
3          \/Iron_Man_(Anthony_%22Tony%22_Stark)
4                          \/Thor_(Thor_Odinson)
                          ...                   
16371                     \/Ru%27ach_(Earth-616)
16372        \/Thane_(Thanos%27_son)_(Earth-616)
16373            \/Tinkerer_(Skrull)_(Earth-616)
16374           \/TK421_(Spiderling)_(Earth-616)
16375                    \/Yologarch_(Earth-616)
Name: urlslug, Length: 16376, dtype: object

In [8]:
url = 'https://marvel.fandom.com/wiki/'
# df['urlslug'].replace('([^\/]+)\/', url, regex=True)
df.update(df['urlslug'].replace('([^\/]+)\/', url, regex=True))
df['urlslug']

0        https://marvel.fandom.com/wiki/Spider-Man_(Pet...
1        https://marvel.fandom.com/wiki/Captain_America...
2        https://marvel.fandom.com/wiki/Wolverine_(Jame...
3        https://marvel.fandom.com/wiki/Iron_Man_(Antho...
4        https://marvel.fandom.com/wiki/Thor_(Thor_Odin...
                               ...                        
16371    https://marvel.fandom.com/wiki/Ru%27ach_(Earth...
16372    https://marvel.fandom.com/wiki/Thane_(Thanos%2...
16373    https://marvel.fandom.com/wiki/Tinkerer_(Skrul...
16374    https://marvel.fandom.com/wiki/TK421_(Spiderli...
16375    https://marvel.fandom.com/wiki/Yologarch_(Eart...
Name: urlslug, Length: 16376, dtype: object

- For the column align, we'd like for the "Characters" part to be removed. We'd like for that column to be categorical (Good, Neutral, Evil or blank).

In [9]:
align = df['ALIGN']
align

0           Good Characters
1           Good Characters
2        Neutral Characters
3           Good Characters
4           Good Characters
                ...        
16371        Bad Characters
16372       Good Characters
16373        Bad Characters
16374    Neutral Characters
16375        Bad Characters
Name: ALIGN, Length: 16376, dtype: object

In [10]:
align.str.replace(" Characters","")

0           Good
1           Good
2        Neutral
3           Good
4           Good
          ...   
16371        Bad
16372       Good
16373        Bad
16374    Neutral
16375        Bad
Name: ALIGN, Length: 16376, dtype: object

- Same for the columns `EYE`, `HAIR`, `SEX` and `ID`. We want to drop the extraneous data and have simple categories (e.g. for `ID` we want categories ["Known","Public", "Secret", "None"] and for `SEX` we just want ["Male", "Female","Genderfluid","Agender"]. There's a way shown below that shows the various possible categories.

In [11]:
df[["EYE", "HAIR", "SEX", "ID"]]

,EYE,HAIR,SEX,ID
0,Hazel Eyes,Brown Hair,Male Characters,Secret Identity
1,Blue Eyes,White Hair,Male Characters,Public Identity
2,Blue Eyes,Black Hair,Male Characters,Public Identity
3,Blue Eyes,Black Hair,Male Characters,Public Identity
4,Blue Eyes,Blond Hair,Male Characters,No Dual Identity
...,...,...,...,...
16371,Green Eyes,No Hair,Male Characters,No Dual Identity
16372,Blue Eyes,Bald,Male Characters,No Dual Identity
16373,Black Eyes,Bald,Male Characters,Secret Identity
16374,NaN,NaN,Male Characters,Secret Identity


In [12]:
idCol = df['ID'].str.replace("Identity","")
idCol

0         Secret 
1         Public 
2         Public 
3         Public 
4        No Dual 
           ...   
16371    No Dual 
16372    No Dual 
16373     Secret 
16374     Secret 
16375         NaN
Name: ID, Length: 16376, dtype: object

In [13]:
df.update(idCol)
# df

In [14]:
df[["EYE", "HAIR", "SEX", "ID"]]

,EYE,HAIR,SEX,ID
0,Hazel Eyes,Brown Hair,Male Characters,Secret
1,Blue Eyes,White Hair,Male Characters,Public
2,Blue Eyes,Black Hair,Male Characters,Public
3,Blue Eyes,Black Hair,Male Characters,Public
4,Blue Eyes,Blond Hair,Male Characters,No Dual
...,...,...,...,...
16371,Green Eyes,No Hair,Male Characters,No Dual
16372,Blue Eyes,Bald,Male Characters,No Dual
16373,Black Eyes,Bald,Male Characters,Secret
16374,NaN,NaN,Male Characters,Secret


- Some data is in the wrong format. For example, `Year` and `APPEARANCES` are floats but they really should be ints (as you cannot have a non-int number of appearances). 

In [15]:
yearCol = df["Year"].fillna(0.0).astype("Int64")
df.update(yearCol)
df["Year"]

0        1962
1        1941
2        1974
3        1963
4        1950
         ... 
16371       0
16372       0
16373       0
16374       0
16375       0
Name: Year, Length: 16376, dtype: object

- Some data doesn't match up (like the `FIRST APPEARANCE` and `Year` columns might have not be the same year)

In [16]:
 df[["FIRST APPEARANCE","Year"]]

,FIRST APPEARANCE,Year
0,Aug-62,1962
1,Mar-41,1941
2,Oct-74,1974
3,Mar-63,1963
4,Nov-50,1950
...,...,...
16371,NaN,0
16372,NaN,0
16373,NaN,0
16374,NaN,0


In [17]:
first_appear = df["FIRST APPEARANCE"].replace(np.nan, 0)
df.update(first_appear)
df["FIRST APPEARANCE"]

0        Aug-62
1        Mar-41
2        Oct-74
3        Mar-63
4        Nov-50
          ...  
16371         0
16372         0
16373         0
16374         0
16375         0
Name: FIRST APPEARANCE, Length: 16376, dtype: object

In [18]:
df["Year"].astype(str).str[0:2]

0        19
1        19
2        19
3        19
4        19
         ..
16371     0
16372     0
16373     0
16374     0
16375     0
Name: Year, Length: 16376, dtype: object

In [19]:
df["APPEARANCE YEAR"] = df["Year"].astype(str).str[0:2] + df["FIRST APPEARANCE"].str.split("-", n = 1, expand = True)[1]
df["APPEARANCE YEAR"].astype('float').astype("Int64")

0        1962
1        1941
2        1974
3        1963
4        1950
         ... 
16371    <NA>
16372    <NA>
16373    <NA>
16374    <NA>
16375    <NA>
Name: APPEARANCE YEAR, Length: 16376, dtype: Int64

In [20]:
appearCol = df["APPEARANCE YEAR"].fillna(0.0).astype(int)
df.update(appearCol)
df["APPEARANCE YEAR"]

0        1962
1        1941
2        1974
3        1963
4        1950
         ... 
16371       0
16372       0
16373       0
16374       0
16375       0
Name: APPEARANCE YEAR, Length: 16376, dtype: object

In [21]:
df.head(20)

,page_id,name,urlslug,ID,ALIGN,EYE,HAIR,SEX,GSM,ALIVE,APPEARANCES,FIRST APPEARANCE,Year,APPEARANCE YEAR
0,1678,Spider-Man (Peter Parker),https://marvel.fandom.com/wiki/Spider-Man_(Pet...,Secret,Good Characters,Hazel Eyes,Brown Hair,Male Characters,NaN,Living Characters,4043.0,Aug-62,1962,1962
1,7139,Captain America (Steven Rogers),https://marvel.fandom.com/wiki/Captain_America...,Public,Good Characters,Blue Eyes,White Hair,Male Characters,NaN,Living Characters,3360.0,Mar-41,1941,1941
2,64786,"Wolverine (James \""Logan\"" Howlett)",https://marvel.fandom.com/wiki/Wolverine_(Jame...,Public,Neutral Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,3061.0,Oct-74,1974,1974
3,1868,"Iron Man (Anthony \""Tony\"" Stark)",https://marvel.fandom.com/wiki/Iron_Man_(Antho...,Public,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,2961.0,Mar-63,1963,1963
4,2460,Thor (Thor Odinson),https://marvel.fandom.com/wiki/Thor_(Thor_Odin...,No Dual,Good Characters,Blue Eyes,Blond Hair,Male Characters,NaN,Living Characters,2258.0,Nov-50,1950,1950
5,2458,Benjamin Grimm (Earth-616),https://marvel.fandom.com/wiki/Benjamin_Grimm_...,Public,Good Characters,Blue Eyes,No Hair,Male Characters,NaN,Living Characters,2255.0,Nov-61,1961,1961
6,2166,Reed Richards (Earth-616),https://marvel.fandom.com/wiki/Reed_Richards_(...,Public,Good Characters,Brown Eyes,Brown Hair,Male Characters,NaN,Living Characters,2072.0,Nov-61,1961,1961
7,1833,Hulk (Robert Bruce Banner),https://marvel.fandom.com/wiki/Hulk_(Robert_Br...,Public,Good Characters,Brown Eyes,Brown Hair,Male Characters,NaN,Living Characters,2017.0,May-62,1962,1962
8,29481,Scott Summers (Earth-616),https://marvel.fandom.com/wiki/Scott_Summers_(...,Public,Neutral Characters,Brown Eyes,Brown Hair,Male Characters,NaN,Living Characters,1955.0,Sep-63,1963,1963
9,1837,Jonathan Storm (Earth-616),https://marvel.fandom.com/wiki/Jonathan_Storm_...,Public,Good Characters,Blue Eyes,Blond Hair,Male Characters,NaN,Living Characters,1934.0,Nov-61,1961,1961


In [22]:
df[["APPEARANCE YEAR","Year"]]

,APPEARANCE YEAR,Year
0,1962,1962
1,1941,1941
2,1974,1974
3,1963,1963
4,1950,1950
...,...,...
16371,0,0
16372,0,0
16373,0,0
16374,0,0


In [23]:
df["APPEARANCE YEAR"].astype(int).dtypes

dtype('int32')

In [24]:
df["Year"].astype(int).dtypes

dtype('int32')

In [25]:
appear = df.loc[(df["APPEARANCE YEAR"] > 0) == df["Year"].notnull(), ["FIRST APPEARANCE","Year", "APPEARANCE YEAR"]]
appear

,FIRST APPEARANCE,Year,APPEARANCE YEAR
0,Aug-62,1962,1962
1,Mar-41,1941,1941
2,Oct-74,1974,1974
3,Mar-63,1963,1963
4,Nov-50,1950,1950
...,...,...,...
16171,Mar-13,2013,2013
16172,Jan-13,2013,2013
16173,Feb-13,2013,2013
16174,Apr-13,2013,2013


In [26]:
appear.shape

(15561, 3)

- Some columns are unnecessary. Some are missing data. Some rows are duplicates. Fix those as necessary.

In [ ]:
# Don't understand

- Some column names are written as uppercase, some are lowercase and some are mixed case. Normalize them so all the column names are in the same format.

In [28]:
df.columns = map(str.lower, df.columns)
df

,page_id,name,urlslug,id,align,eye,hair,sex,gsm,alive,appearances,first appearance,year,appearance year
0,1678,Spider-Man (Peter Parker),https://marvel.fandom.com/wiki/Spider-Man_(Pet...,Secret,Good Characters,Hazel Eyes,Brown Hair,Male Characters,NaN,Living Characters,4043.0,Aug-62,1962,1962
1,7139,Captain America (Steven Rogers),https://marvel.fandom.com/wiki/Captain_America...,Public,Good Characters,Blue Eyes,White Hair,Male Characters,NaN,Living Characters,3360.0,Mar-41,1941,1941
2,64786,"Wolverine (James \""Logan\"" Howlett)",https://marvel.fandom.com/wiki/Wolverine_(Jame...,Public,Neutral Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,3061.0,Oct-74,1974,1974
3,1868,"Iron Man (Anthony \""Tony\"" Stark)",https://marvel.fandom.com/wiki/Iron_Man_(Antho...,Public,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,2961.0,Mar-63,1963,1963
4,2460,Thor (Thor Odinson),https://marvel.fandom.com/wiki/Thor_(Thor_Odin...,No Dual,Good Characters,Blue Eyes,Blond Hair,Male Characters,NaN,Living Characters,2258.0,Nov-50,1950,1950
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16371,657508,Ru'ach (Earth-616),https://marvel.fandom.com/wiki/Ru%27ach_(Earth...,No Dual,Bad Characters,Green Eyes,No Hair,Male Characters,NaN,Living Characters,NaN,0,0,0
16372,665474,Thane (Thanos' son) (Earth-616),https://marvel.fandom.com/wiki/Thane_(Thanos%2...,No Dual,Good Characters,Blue Eyes,Bald,Male Characters,NaN,Living Characters,NaN,0,0,0
16373,695217,Tinkerer (Skrull) (Earth-616),https://marvel.fandom.com/wiki/Tinkerer_(Skrul...,Secret,Bad Characters,Black Eyes,Bald,Male Characters,NaN,Living Characters,NaN,0,0,0
16374,708811,TK421 (Spiderling) (Earth-616),https://marvel.fandom.com/wiki/TK421_(Spiderli...,Secret,Neutral Characters,NaN,NaN,Male Characters,NaN,Living Characters,NaN,0,0,0
